<div class="container" style="width: 100%; height: 200px; overflow: hidden; text-align: left">
    <!-- image source: ??????? -->
<!--     <img src="../images/center-pivot-irrigation-aerial.png" style="opacity: 0.4; margin: -350px; object-fit: fill;"></img> -->
    <div class="title-text" style="color: black; position: absolute; left: 16px; top: 20%; font-size: 3em; font-weight: bold;">
        Life Cycle of a Fire
    </div>
    <br>
    <div style="color: black; position: absolute; left: 20px; top: 50%; font-size: 1.5em; font-weight: bold;">
        NASA EIS Transdisciplinary Science from Pre-Fire to Post-Fire
    </div>
</div>

## Introduction

Fires are an integral part of the Earth system and have a large human dimension. Society's interaction with the Earth system affects fire risk, fire spread, and fire impacts (e.g. property destruction, emissions, mud slides). NASA data products and models provide real-time information on active fires, smoke, air quality, and forecasts of changing fire weather and seasonal fire risk.

EIS provides a platform for collaborative analysis of and decision support for pre-fire risk, active fire situational awareness, and managing post-fire impacts. The full potential of these individual efforts are realized through the EIS collaborative framework to support new transdisciplinary connections and fire-related management activities.

## Fires in Western US

The Western United States is experiencing an increase of catastrophic wildfires. While fires historically constituted an important rangeland disturbance process in the semi-arid ecosystems of the Western US, climate change and intensifying droughts have increased the frequency and vigor of wildfire occurrences.


In this notebook we explore impacts on vegetation due to fires in Western US, using evapotranspiration data from OpenET product and that modeled using [NASA's Land Information System](https://lis.gsfc.nasa.gov/) (LIS) modelling framework.

<!-- ## Overview of LIS Simulations

| Name      | Resolution | Run Period | Land Surface Model | Data Assimilated                                                    | Meteorological Forcing |
|-----------|------------|------------|--------------------|---------------------------------------------------------------------|------------------------|
| Open Loop | 10km       | 2001-2021  | NoahMP-4.0.1       | None                                                                | GPM IMERG              |
| Multi-DA  | 10km       | 2001-2021  | NoahMP-4.0.1       | Leaf Area Index (MODIS), Terrestrial Water Storage (GRACE/GRACE-FO) | GPM IMERG              |

<br> -->

<!-- <center>
    <img src="../images/GWpumping_StoryMap_simulation_overview.png" style="width: 50%; border-radius:10px">
</center> -->

In [ ]:
############################################
#              Import libraries            #
############################################

# import s3fs
# import intake
# from eis_freshwater_py.config import intake_catalog_path
# from eis_freshwater_py.lis_output import add_latlon_coords, clip_to_shape
# import geopandas as gpd
# from matplotlib import pyplot as plt
# import numpy as np
# import pandas as pd
# import xarray as xr
import rioxarray as rxr
# import holoviews as hv
import geoviews as gv
import panel as pn
# import pandas as pd

from holoviews import opts
# from holoviews.streams import Stream, param
# import datashader
# from holoviews.operation.datashader import rasterize

import hvplot.pandas
# import hvplot.xarray

import pandas as pd
# import pandas_bokeh
# import matplotlib.pyplot as plt
# import matplotlib.dates as dts
# from matplotlib.patches import Patch
# from matplotlib.lines import Line2D
# import numpy as np 

# show/suppress runtime warnings
import warnings
# uncomment next line to suppress runtime warnings (USE DURING DEMO)
warnings.filterwarnings('ignore')

# s3 = s3fs.S3FileSystem(anon=False)
# hv.extension('bokeh', logo=False)

## Is post-fire vegetation response captured by LSM assimilating remote sensing observed disturbances?

In [ ]:
## LIS processed tiffs
map_overlay =  gv.tile_sources.EsriImagery  

et_map_opts = opts.Image(width=400, height=400, cmap='YlGn',
                                   colorbar=True, clim=(0, 0.01), clabel='Annual ET (mm)',
                                   tools=['hover'])

etimg20 = gv.load_tiff("./DATA/lis_et2020.tiff", nan_nodata=True)
gv.tile_sources.OSM() * etimg20.opts(et_map_opts).opts(title="LIS-ET WY2020 (mm)") * map_overlay
# etimg20

# etimg21 = gv.load_tiff("./DATA/lis_et2021.tiff", nan_nodata=True)
# gv.tile_sources.OSM() * etimg21.opts(et_map_opts).opts(title="LIS-ET WY2021 (mm)") * map_overlay
# etimg21

map_row = pn.Row(pn.Column(etimg20))

pn.Column(  map_row)

In [ ]:
## ET suppression and Burn Scars

map_overlay =  gv.tile_sources.EsriImagery  
WD = 380
HT = 380
etsupp_map_opts = opts.Image(width=WD, height=HT, cmap='coolwarm_r',
                                   colorbar=True, clim=(-150,150), clabel='ET Change (mm)',
                                   tools=['hover'])
tvegsupp_map_opts = opts.Image(width=WD, height=HT, cmap='coolwarm_r',
                                   colorbar=True, clim=(-150,150), clabel='TVeg Change (mm)',
                                   tools=['hover'])
gppsupp_map_opts = opts.Image(width=WD, height=HT, cmap='coolwarm_r',
                                   colorbar=True, clim=(-600,600), clabel='GPP Change (mm)',
                                   tools=['hover'])
mtbs_map_opts = opts.Image(width=WD, height=HT, cmap='inferno_r',
                                   colorbar=True, clim=(0,5), clabel='Severity',
                                   tools=['hover'])
### Total ET
et20 = rxr.open_rasterio("./DATA/lis_et2020_CA.tiff", masked=True).squeeze()
et21 = rxr.open_rasterio("./DATA/lis_et2021_CA.tiff", masked=True).squeeze()
etsupp = (et21 - et20)*86400 #/et20 #/(geotiff_da)
etsuppDS = gv.Dataset(etsupp) #_ds, ['x', 'y'], 'et')
etsuppimg = etsuppDS.to(gv.Image)
gv.tile_sources.OSM() * etsuppimg.opts(etsupp_map_opts).opts(title="LIS-ET Suppression WY21-20") #* map_overlay

### TVEG
et20 = rxr.open_rasterio("./DATA/lis_tveg2020_CA.tiff", masked=True).squeeze()
et21 = rxr.open_rasterio("./DATA/lis_tveg2021_CA.tiff", masked=True).squeeze()
etsupp = (et21 - et20)*86400 #/et20 #/(geotiff_da)
etsuppDS = gv.Dataset(etsupp) #_ds, ['x', 'y'], 'et')
tvegsuppimg = etsuppDS.to(gv.Image)
gv.tile_sources.OSM() * tvegsuppimg.opts(tvegsupp_map_opts).opts(title="LIS-TVeg Suppression WY21-20") #* map_overlay

### GPP
# et20 = rxr.open_rasterio("./DATA/lis_gpp2020.tiff", masked=True).squeeze()
# et21 = rxr.open_rasterio("./DATA/lis_gpp2021.tiff", masked=True).squeeze()
# etsupp = (et21 - et20)*86400 #/et20 #/(geotiff_da)
# etsuppDS = gv.Dataset(etsupp) #_ds, ['x', 'y'], 'et')
# gppsuppimg = etsuppDS.to(gv.Image)
# gv.tile_sources.OSM() * gppsuppimg.opts(gppsupp_map_opts).opts(title="LIS-GPP Suppression WY21-20") #* map_overlay


mtbs20 = gv.load_tiff("./DATA/mtbs2020_CA.tiff", nan_nodata=True)
gv.tile_sources.OSM() *mtbs20.opts(mtbs_map_opts).opts(title="MTBS Burn Severity 2020") # * map_overlay



map_row = pn.Row(pn.Column(etsuppimg),
                 pn.Column(tvegsuppimg),
#                  pn.Column(gppsuppimg),
                 pn.Column(mtbs20)
                )

pn.Column(map_row)

## Conclusions

* Recovery of vegetation from fire disturbances is a complex interplay of climate conditions, landcover, and burn severity.

* Under drought conditions, grasslands are impacted the most while forests can sustain damage even in near normal or wet conditions. 


* Action items for hydrologic modeling community and fire-managers – recovery assessment needs to incorporate vegetation response to climate disturbances